# Bus vs Tram delay comparisons

This notebook reads the local database (departures_raw) and plots average delays (in full minutes) over time for BUS vs TRAM, per day and per month.

It looks for a config at ../config/local.yml for the DB URL; otherwise defaults to ../data/reliability.db (SQLite).

In [ ]:
%pip -q install pandas matplotlib seaborn SQLAlchemy PyYAML

In [ ]:
from pathlib import Path
import os, json, yaml
import pandas as pd
from sqlalchemy import create_engine

# Infer project root (this notebook is expected under TrackTramReliablilty/notebooks)
nb_dir = Path.cwd()
proj_root = nb_dir if (nb_dir / 'pyproject.toml').exists() else nb_dir.parent
config_path = proj_root / 'config' / 'local.yml'
default_sqlite = (proj_root / 'data' / 'reliability.db').resolve()
db_url = None
# Env override
db_url = os.getenv('TTR_DB_URL')
if not db_url:
    if config_path.exists():
        cfg = yaml.safe_load(config_path.read_text()) or {}
        db_url = cfg.get('db_url')
    if not db_url:
        db_url = f'sqlite:///{default_sqlite}'

print('Using DB URL:', db_url)
engine = create_engine(db_url)

In [ ]:
import pandas as pd

query = """
SELECT
  date(datetime(fetched_at, 'unixepoch')) AS dt,
  transport_type,
  delay_in_minutes
FROM departures_raw
WHERE transport_type IN ('BUS','TRAM')
  AND delay_in_minutes IS NOT NULL
  AND cancelled = 0
ORDER BY dt ASC
"""
df = pd.read_sql_query(query, engine)
df['dt'] = pd.to_datetime(df['dt'])
df['delay_in_minutes'] = df['delay_in_minutes'].astype(float)
df.shape, df.head()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_theme(style='whitegrid')

# Daily averages (rounded to full minutes)
daily = (
    df.groupby(['dt', 'transport_type'])['delay_in_minutes'].mean().round().reset_index()
)
daily.rename(columns={'delay_in_minutes': 'avg_delay_min'}, inplace=True)

fig, ax = plt.subplots(figsize=(11, 5))
sns.lineplot(data=daily, x='dt', y='avg_delay_min', hue='transport_type', marker='o', ax=ax)
ax.set_title('Average delay per day (BUS vs TRAM)')
ax.set_xlabel('Date')
ax.set_ylabel('Average delay (full minutes)')
ax.legend(title='Transport')
fig.autofmt_xdate()
# Save figure
fig_dir = proj_root / 'notebooks' / 'figures'
fig_dir.mkdir(parents=True, exist_ok=True)
fig_path = fig_dir / 'daily_bus_tram_delays.png'
fig.savefig(fig_path, dpi=150, bbox_inches='tight')
fig_path

In [ ]:
# Monthly averages (rounded to full minutes)
df['month'] = df['dt'].dt.to_period('M').dt.to_timestamp()
monthly = (
    df.groupby(['month', 'transport_type'])['delay_in_minutes'].mean().round().reset_index()
)
monthly.rename(columns={'delay_in_minutes': 'avg_delay_min'}, inplace=True)

fig, ax = plt.subplots(figsize=(11, 5))
sns.lineplot(data=monthly, x='month', y='avg_delay_min', hue='transport_type', marker='o', ax=ax)
ax.set_title('Average delay per month (BUS vs TRAM)')
ax.set_xlabel('Month')
ax.set_ylabel('Average delay (full minutes)')
ax.legend(title='Transport')
fig.autofmt_xdate()
fig_path2 = fig_dir / 'monthly_bus_tram_delays.png'
fig.savefig(fig_path2, dpi=150, bbox_inches='tight')
fig_path2